# Generate

In [ ]:
!pip install -q Faker

In [24]:
import numpy as np
import pandas as pd
import random
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from faker import Faker
faker = Faker()
from faker.providers import DynamicProvider

In [38]:
def ccas_cat(n, categories, ratios=None):
    if ratios is None:
        ratios = [1/len(categories)]*len(categories)
    return (np.random.choice(categories, size=(n,), p=[float(i)/sum(ratios) for i in ratios])).tolist()

def ccas_cont(n, dist, a, b, maxn=1, minn=0):
    if dist=='norm':
        return [int(i) for i in (np.random.normal(a,b,n))]
    elif dist=='beta':
        return [int(i) for i in (minn + (np.random.beta(a, b, n) * (maxn - minn)))]
    elif dist=='weibull':
        return [int(i) for i in (minn + (np.random.weibull(a, n) * (maxn - minn)))]

def randomDate(start, end):
    frmt = '%d-%m-%Y %H:%M'
    stime = time.mktime(time.strptime(start, frmt))
    etime = time.mktime(time.strptime(end, frmt))
    ptime = stime + random.random() * (etime - stime)
    dt = datetime.fromtimestamp(time.mktime(time.localtime(ptime)))
    return dt

In [27]:
seed = 8203
np.random.seed(seed)
faker.seed_instance(seed)

In [ ]:
# medical_professions_provider = DynamicProvider(
#      provider_name="medical_profession",
#      elements=["dr.", "doctor", "nurse", "surgeon", "clerk"],
# )
# faker.add_provider(medical_professions_provider)
# faker.medical_profession()

In [29]:
p = 10
df = pd.DataFrame()
df['client_id'] = [faker.ssn() for i in range(p)]
df['first_name'] = [faker.first_name() for i in range(p)]
df['last_name'] = [faker.last_name() for i in range(p)]
df['city'] = [faker.city() for i in range(p)]
df['phone'] = [faker.phone_number() for i in range(p)]
df['email'] = df.apply(lambda row: "{}.{}@gmail.com".format(row.first_name.lower(),
                                                            row.last_name.lower()), axis=1)
clients = df.copy()
clients

,client_id,first_name,last_name,city,phone,email
0,154-64-9693,Sarah,Garcia,Danielleborough,(622)537-2935x31417,sarah.garcia@gmail.com
1,148-49-3184,Lucas,Clark,Wendyfurt,+1-271-568-2657x84023,lucas.clark@gmail.com
2,488-22-2850,Mark,Nelson,Zamoraburgh,309-598-7241,mark.nelson@gmail.com
3,898-73-3339,Edward,King,Port Jeffreymouth,+1-035-807-5780,edward.king@gmail.com
4,830-11-8837,James,Camacho,Curtisborough,+1-459-922-7767x59844,james.camacho@gmail.com
5,574-92-3858,Amy,Phillips,Kimberlyland,+1-104-625-7936x560,amy.phillips@gmail.com
6,877-33-4462,Scott,Burns,East Saraport,(004)438-4375,scott.burns@gmail.com
7,431-25-4334,Elizabeth,Brown,Port Tyler,114.111.7527,elizabeth.brown@gmail.com
8,594-87-8512,Michael,Stephens,North Arthur,(357)296-5861x2781,michael.stephens@gmail.com
9,038-37-7264,Johnny,Smith,Port Kathleen,534.313.2507x2845,johnny.smith@gmail.com


In [31]:
p = 3
df = pd.DataFrame()
df['therapist_id'] = [faker.ssn() for i in range(p)]
df['first_name'] = [faker.first_name() for i in range(p)]
df['last_name'] = [faker.last_name() for i in range(p)]
therapists = df.copy()
therapists.head()

,therapist_id,first_name,last_name
0,030-45-1969,Donald,Dean
1,280-65-5827,Matthew,Parrish
2,150-60-1665,Nichole,Garcia


In [33]:
p = 5
df = pd.DataFrame()
df['service_id'] = np.arange(0, p, 1).tolist()
df['service_name'] = ['speech therapy', 'dietetics', 'physiotherapy', 'occupational therapy', 'music therapy']
services = df.copy()
services.head()

,service_id,service_name
0,0,speech therapy
1,1,dietetics
2,2,physiotherapy
3,3,occupational therapy
4,4,music therapy


In [57]:
p = 10000
df = pd.DataFrame()
df['visit_id'] = np.arange(0, p, 1).tolist()
df['client_id'] = ccas_cat(p, clients.client_id.tolist())
df['therapist_id'] = ccas_cat(p, therapists.therapist_id.tolist())
df['service_id'] = ccas_cat(p, services.service_id.tolist())
df['visit_ts'] = [faker.date_time_between_dates(datetime_start=datetime(2022,1,1),
                                                       datetime_end=datetime(2022,6,30))\
                         .strftime("%Y/%m/%d, %H:%M:%S") for i in range(p)]
visits = df.copy()
visits.head()

,visit_id,client_id,therapist_id,service_id,visit_ts
0,0,830-11-8837,150-60-1665,3,"2022/05/21, 09:34:39"
1,1,898-73-3339,150-60-1665,1,"2022/02/11, 23:51:36"
2,2,154-64-9693,030-45-1969,0,"2022/03/23, 15:21:37"
3,3,148-49-3184,030-45-1969,1,"2022/03/09, 17:27:23"
4,4,148-49-3184,150-60-1665,3,"2022/05/08, 09:32:03"


In [64]:
visits.to_csv("visits.csv", index=False)
therapists.to_csv("therapists.csv", index=False)
clients.to_csv("clients.csv", index=False)
services.to_csv("services.csv", index=False)

## Load to Database

In [58]:
%%capture
pip install psycopg2-binary

In [59]:
host="database-1.<>.us-east-1.rds.amazonaws.com"
user="postgres"
password=""

In [60]:
import os
import glob
import psycopg2
import pandas as pd
import sqlalchemy as sa
import warnings
warnings.filterwarnings('ignore')

In [61]:
def create_database():
    """
    - Creates and connects to the clinic
    - Returns the connection and cursor to clinic
    """
    
    # connect to default database
    conn = psycopg2.connect(f"host={host} dbname=template1 user={user} password={password}")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create clinic database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS clinic WITH (FORCE)")
    cur.execute("CREATE DATABASE clinic WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
    conn.close()
    
    # connect to clinic database
    conn = psycopg2.connect(f"host={host} dbname=clinic user={user} password={password}")
    cur = conn.cursor()
    
    return cur, conn 

In [62]:
_, _ = create_database()

In [63]:
db_url = f'postgresql://{user}:{password}@{host}:5432/clinic'
engine = sa.create_engine(db_url)

In [ ]:
visits.to_sql('visits', engine, method='multi', index=False, if_exists='replace')
therapists.to_sql('therapists', engine, method='multi', index=False, if_exists='replace')
clients.to_sql('clients', engine, method='multi', index=False, if_exists='replace')
services.to_sql('services', engine, method='multi', index=False, if_exists='replace')